In [2]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import math
import os
from pydub import AudioSegment

# My audio selection
Vengono selezionati solo audio che hanno durata >= 3 secondi e che hanno file audio presente nel dataset

In [3]:
bird_tags = scipy.io.loadmat('Bird_tags_Train.mat')["Bird_tags"]
coords = {}
inverted_coords = {}
audio_path = "D:\\Giacomo\\Tovanella-20241110T120546Z-001\\Tovanella"
for elem in bird_tags:
    tag = elem[0][0][0][0][0]
    tag = tag.replace("_", " ")
    file_name = elem[0][0][0][1][0]
    file_path = os.path.join(audio_path, file_name)
    bbox = elem[0][0][0][3][:4]
    start_time = math.floor(min(bbox[:, 0]))
    end_time = math.floor(max(bbox[:, 0]))
    duration = end_time - start_time
    if not os.path.exists(file_path) or duration < 3:   # la selection
        continue
    if tag not in coords:
        coords[tag] = []
    if file_name not in inverted_coords:
        inverted_coords[file_name] = []
    coords[tag].append({"file_name": file_name, "bbox": bbox.tolist(), "start_time": start_time, "duration": duration})
    inverted_coords[file_name].append({"tag": tag, "start_time": start_time, "duration": duration})


In [4]:
# generate json file of coords
import json
with open("utils/coords.json", "w") as f:
    json.dump(coords, f)
with open("utils/inverted_coords.json", "w") as f:
    json.dump(inverted_coords, f)